# Teeview Analytics

The purpose of this project is to build a simple analytics tool that allows us to keep a tab on the Teespring campaigns indexed by teeview that have potential to be successfull in order to get inspiration only from this subset. 

The final outcome should initially be a bot (think Slack) that informs us when a new potential successfull campaign is identified in order to jump on it right away and draw inspiration from it. The purpose of the bot is to get near real time notifications since in this cases timing is fundamental.

The principles that will be used to identify the campaigns that have success potential which are the ones we need to draw attention to is basic physics principles. We will get the velocity of the campaign and its acceleration using the numerical derivative. This will produce 3 plots which are the position, velocity and acceleration of a campaign. While a successfull campaign has a good final position it should have had high velocity values and high acceleration values to reach such velocities. This is the case because the length of the campaigns is fixed. That is the equivalent to having a sprint in runners where there is a defined distance beforehand.

Given this context the goal is to identify potentially successfull campaigns i.e. noticing campaigns that have the potential to be successfull before they actually succeed (since one they succeed the market will be saturated of that design and as such it will no longer represent an opportunity).

In order to make this identification one needs to check the acceleration value in conjunction with the velocity. Think that there are at least 2 cases in which you would definitely would want to get a piece of the action.

1. Acceleration is practically null or even negative but velocity is still quite high.
2. Velocity is still slow but acceleration is really high.

There should be a coefficient that would indicate if we should jump on a campaign or not. To be developed.

#### Crawler

First step is to crawl [https://www.teeview.org] and get the latest campaigns. Let's define the "latest campaigns" as the ones which have been added during the last day. We should only select those which say:

1. "31 sold, available until [Thursday!]"
2. "Only 3 more needed to print!" 

In order to do this we need to get the url of the campaign and go to [http://www.teespring.com] to see if this message appears on the campaign. If it does then we have data to populate our database and this url goes into the list of urls which we will follow periodically along with the time ago it was added, we will run the script periodically to get the sales data and be able to make the position, velocity and acceleration plots.

#### Import Modules

In [41]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import json
import time
import sys

#### Create Database Tables

There should be 2 database tables. 

One of them will store the information of relevant campaigns, a relevant campaign is defined as that which is obtained through the "teeview_scraper" function and posteriorly filtered through the "campaigns_updater" function (which gets all campaigns that are added within one day ago and filters only the ones which report sales data. This table will be called "Campaigns"

The second database table will consist of the entries produced by the "sales_data_updater" function which will query Teespring for each campaign in "Campaigns" and add the latest sales data as an entry to the table. This will be the data that will be used to plot position, velocity and acceleration. This table will be called "SalesData"

The database as a whole should be cleaned every day or so to keep it short and running smoothly. The cleaning process will consist of going through the database to see which campaigns have ended and removing the respective rows from the "Campaign" and "SalesData"   

In [32]:
# This function creates a database instance which consists of 2 tables. One called "campaigns"
# which stores the information of new and active campaigns which report sales data and one
# called "sales_data" which stores the sales data information of such campaigns
def create_database():
    print("started creating database...")
    # Connect to "teeview_analytics" database
    conn = sqlite3.connect('teeview_analytics.db')
    # Create "campaigns" table if it does not exist
    campaigns_table = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='campaigns'").fetchall()
    if len(campaigns_table) == 0: conn.execute("create table campaigns(url, name, img)")
    # Create "sales_data" table if it does not exist
    campaigns_table = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sales_data'").fetchall()
    if len(campaigns_table) == 0: conn.execute("create table sales_data(campaign_url, sales, timestamp)")
    # Close connection
    conn.close()
    print("finished creating database!")

In [33]:
# This function will clean the database removing the entries in both tables i.e. "campaigns" and
# "sales_data" which are associated to campaigns that have already ended
def clean_database():
    print("starting cleaning database...")
    # Connect to "teeview_analytics" database
    conn = sqlite3.connect('teeview_analytics.db')
    # Loop through "campaigns" table
    for row in conn.execute("SELECT url FROM campaigns").fetchall():
        campaign_url = row[0]
        response = requests.get(campaign_url)
        html = BeautifulSoup(response.text, "html.parser")
        # Text of relevant html elements of campaign
        sales_data_text = html.select(".persistent_timer__order_count")[0].getText().lower()
        ended_campaign_text = html.select(".persistent_timer__stats--relaunchable")
        # Delete associated rows in "campaigns" and "sales_data" tables if campaign has ended
        if ended_campaign_text or ("only" not in sales_data_text and "sold" not in sales_data_text):
            print("DELETE CAMPAIGN WITH URL: {0}".format(campaign_url))
            conn.execute("DELETE FROM campaigns WHERE url='{0}'".format(campaign_url))
            conn.execute("DELETE FROM sales_data WHERE campaign_url='{0}'".format(campaign_url))
    # Commit changes to connection
    conn.commit()
    # Close connection
    conn.close()
    print("finished cleaning database!")

#### Define JSON Config File Read & Update Function

Define function to update configuration JSON file to store the "latest_campaign_link", the "last_query_timestamp" and any other variable that needs to be stored persistently but does not constitute an entry in any of the database tables.

In [34]:
# Updates the config file with the "latest_campaign_link" and the "last_query_timestamp" 
def update_config(latest_campaign_link, last_query_timestamp):
    # Create config data object
    data = {"LATEST_CAMPAIGN_LINK": latest_campaign_link, "LAST_QUERY_TIMESTAMP": last_query_timestamp, "SLEEP_INTERVAL": 3600}
    # Write config data object into config.json
    with open("config.json", "w") as f:
        json.dump(data, f)

In [35]:
# Reads config JSON file and return the Python Object Representation
def read_config():
    with open("config.json", "r") as f:
        return json.load(f)

#### Define Main Functions

On this section the main functions used by the program to fetch the data periodically are defined, namely:

1. **teeview_scraper():** This function is the one responsible for scraping the links out of [http://www.teeview.org] that are within a day ago or which are younger than the youngest link of the last query.
2. **campaigns_updater():** This function has as a parameter the links obtained by the **teeview_scraper()** function and its purpose is to go through each one of these links and only get the appropriate info on those campaigns which report sales data and for each of these campaigns make the appropriate entry to the "campaigns" database table.
3. **sales_data_updater():** This function has no parameters. Is runned after the **campaigns_updater()** function and it loops through the "campaigns" database table and queries such campaigns in order to extract the latest sales reported data and make the appropriate entries to the database.

In [36]:
# This function will make successive get requests to teeview and return the new campaigns that were added
# within the last day or sooner than the latest campaign saved on the last query latest_campaign_link
def teeview_scraper():
    print("starting teeview_scraper...")
    # Set variables of: first "page" to query, query while data is "whithin_day" and array to store "teeview_data" 
    page = 1
    within_day = True
    teeview_data = []
    config_data = read_config()
    # Loop all campaign that were added within one day ago
    while within_day:
        # Form url of teeview made up of active campaigns on page "page" 
        url = "https://www.teeview.org/site/index?active=true&page={0}&per-page=12".format(page)
        # Make request to get the page of teeview and parse it
        response = requests.get(url)
        html = BeautifulSoup(response.text, 'html.parser')
        # Get links of teespring campaigns of the page queried
        for thumbnail in html.select(".thumbnail"):
            campaign_url = thumbnail.select("h3 a")[0]['href']
            campaign_time_ago = thumbnail.select("p.text-muted small")[0].getText()
            # Break the loop if we have reached the campaigns that were added longer than a day ago
            if "day" in campaign_time_ago:
                within_day = False
                break
            # Break the loop if we have reached the latest_campaign_queried
            if config_data["LATEST_CAMPAIGN_LINK"] == campaign_url:
                within_day = False
                break
            # Add the campaign data to teeview_data
            teeview_data.extend([campaign_url])
        # Increase page counter
        page += 1
    print("finished teeview_scraper!")
    # Returns the gathered "teeview_data"
    return teeview_data

In [37]:
# This function will go through the new campaigns returned by the "teeview_scraper" function
# and for each of them request the Teespring campaign in order to find out if it is relevant
# or not based on the fact if the campaign reports sales data or not. If it does it adds the
# new campaign to the respective database table.
def campaigns_updater(teeview_data):
    print("starting capaigns_updater...")
    # Update the JSON config file if there is new data on teeview
    if teeview_data:
        update_config(teeview_data[0], int(time.time()))
    # Define variables
    campaigns = []
    # Loop through each link and check if it reports sales data on Teespring campaign page
    for campaign_url in teeview_data:
        response = requests.get(campaign_url)
        html = BeautifulSoup(response.text, "html.parser")
        # If the html element where sales data is reported exit continue
        sales_data_html_el = html.select(".persistent_timer__order_count")
        if sales_data_html_el:
            sales_data_text = sales_data_html_el[0].getText().lower()
            # If it reports sales data append campaign to "campaigns" variable
            if "only" in sales_data_text or "sold" in sales_data_text:
                campaigns.append((campaign_url, html.select(".campaign__name")[0].getText(), "https:" + html.select(".image_stack__image")[0]["src"]))
    # If new campaigns with reported sales data were found add them to database
    if campaigns:
        # Connect to "teeview_analytics" database
        conn = sqlite3.connect('teeview_analytics.db')
        # Add filtered campaigns data to "campaigns" table
        conn.executemany("INSERT INTO campaigns(url, name, img) VALUES (?, ?, ?)", campaigns)
        # Commit changed to connection
        conn.commit()
        # Close connection
        conn.close()
    print("finished capaigns_updater!")

In [38]:
# This function will go through the database table that stores the Teespring campaigns that
# report sales data and query each of them on Teespring to get the latest info on sales which
# will be stored as a new entry on the database table that stores the sales or "position" data
# of each relevant campaign.
def sales_data_updater():
    print("starting sales_data_updater...")
    # Connect to "teeview_analytics" database
    conn = sqlite3.connect('teeview_analytics.db')
    # Loop through campaigns and update "sales_data" table by making the appropriate entries
    for row in conn.execute("SELECT url FROM campaigns").fetchall():
        # Teespring campaign url
        campaign_url = row[0]
        # Variables
        sales = None
        timestamp = int(time.time())
        # Make entry with the campaign latest sales reported data
        response = requests.get(campaign_url)
        html = BeautifulSoup(response.text, "html.parser")
        sales_data_text = html.select(".persistent_timer__order_count")[0].getText()
        # Get reported sales data depending on wether goal has been achieved or not yet
        if "only" in sales_data_text.lower():
            sales = -int(sales_data_text.lower().split(" ")[1])
        if "sold" in sales_data_text.lower():
            sales = int(sales_data_text.lower().split(" ")[0])
        # Make entry in "sales_data" table if campaign is reporting sales otherwise append to "campaign_urls_to_delete"
        if type(sales) == int: 
            conn.execute("INSERT INTO sales_data VALUES ('{0}', {1}, {2})".format(campaign_url, sales, timestamp))
        else:
            print("ERROR: In sales_data_updater(): Sales reported data is not of type int.")
    # Commit changes to connection
    conn.commit()
    # Close connection
    conn.close()
    print("finished sales_data_updater!")

#### Define Loop Function

The purpose of this section is to assemble it all together and define the loop that will be runned every 2 hours as defined in the JSON config file

In [39]:
# Loop function
def do_loop():
    # Creates a database if it does not exist already
    create_database()
    # Cleans up the database associated entries with campaigns that have expired
    clean_database()
    # Updates the "campaigns" with the result from the "teeview_scraper"
    campaigns_updater(teeview_scraper())
    # Updates the "sales_data" database table
    sales_data_updater()

#### Run Loop

In [ ]:
# Read config data
config_data = read_config()
# Run loop every 2 hours as defined in the config file variable "SLEEP_INTERVAL"
while True:
    print("{}: Starting scrape cycle".format(time.ctime()))
    try:
        do_loop()
    except KeyboardInterrupt:
        print("Exiting....")
        sys.exit(1)
    except Exception as exc:
        print("Error with the scraping:", sys.exc_info()[0])
        traceback.print_exc()
    else:
        print("{0}: Successfully finished scraping".format(time.ctime()))
        time.sleep(config_data["SLEEP_INTERVAL"])

Thu Feb 23 07:47:13 2017: Starting scrape cycle
started creating database...
finished creating database!
starting cleaning database...
